In [8]:
# 有shock發生時
import numpy as np
from numpy.linalg import inv
class Demand:
    def __init__(self, D, h):
        self.D=D
        self.h=h
    def quantity(self, pd):
        qd=np.array((self.D@pd)+self.h)
        return qd
class Supply:
    def __init__(self, C, e):
        self.C=C
        self.e=e
    def quantity(self, ps):
        qs=np.array((self.C@ps)+self.e)
        return qs 
class Market:
    def __init__(self, demand, supply):
        self.demand=demand
        self.supply=supply
    def price(self):
        pe=inv(self.supply.C-self.demand.D)@(self.demand.h-self.supply.e)
        return pe
    def quantity(self):
        qe=self.demand.D@self.price()+self.demand.h
        return qe
    def equlibrium(self):
        pe=self.price()
        qe=self.quantity()
        return pe, qe

D=np.array([
    [-10,-5],
    [-1,-10]
])
h=np.array([
    [100],
    [50]
])
C=np.array([
    [10,5],
    [5,10]
])
e=np.array([
    [0],
    [0]
])

demand = Demand(D, h)
supply = Supply(C, e)
market = Market(demand, supply)

market.equlibrium()

(array([[4.41176471],
        [1.17647059]]),
 array([[50.        ],
        [33.82352941]]))

In [11]:
# 有shock發生時
import numpy as np
from numpy.linalg import inv
class Demand:
    def __init__(self, D, h):
        self.D=D
        self.h=h
    def quantity(self, pd):
        qd=np.array((self.D@pd)+self.h)
        return qd
class Supply:
    def __init__(self, C, e):
        self.C=C
        self.e=e
    def quantity(self, ps):
        qs=np.array((self.C@ps)+self.e)
        return qs 
class Market:
    def __init__(self, demand, supply):
        self.demand=demand
        self.supply=supply
    def price(self):
        D=self.demand.D
        h=self.demand.h
        C=self.supply.C
        e=self.supply.e
        pe=inv(C-D)@(h-e)
        return pe
    def quantity(self):
        D=self.demand.D
        qe=D@self.price()+self.demand.h
        return qe
    def equilibrium(self):
        pe=self.price()
        qe=self.quantity()
        return pe, qe

D=np.array([
    [-10,-5],
    [-1,-10]
])
h=np.array([
    [100],
    [50]
])
C=np.array([
    [10,5],
    [5,10]
])
e=np.array([
    [0],
    [0]
])

demand = Demand(D, h)
supply = Supply(C, e)
market = Market(demand, supply)

market.equilibrium()

(array([[4.41176471],
        [1.17647059]]),
 array([[50.        ],
        [33.82352941]]))

In [12]:
# 改變參數
demand.h = np.array([[80],[50]])
market.equilibrium()

(array([[3.23529412],
        [1.52941176]]),
 array([[40.        ],
        [31.47058824]]))

In [18]:
# Overlapping generation model
# Consumer: Log Utility
class Consumer:
    def __init__(self, beta):
        self.beta=beta
    def foc_c(self, knext, w):
        """
        s=knext
        """
        return knext-(self.beta/(1+self.beta))*w
# Producer: Cobb-Douglas
class Producer:
    def __init__(self, alpha):
        self.alpha=alpha
    def foc_l(self, k, w, l=1):
        return (1-self.alpha)*(k/l)**(self.alpha)-w
    def foc_k(self, k, R, l=1):
        return self.alpha*(l/k)**(1-self.alpha)-R
# Dynamic equilibrium
"""
satisfies all FOCs
satisfies all constraints
Assume l=1 for all t, and k0 is given.
"""
class OLGMarket:
    def __init__(self, consumer, producer, k0):
        self.consumer = consumer
        self.producer = producer
        self.k0 = k0
        self.k = k0
        self.equilibrium_dynamics = [[k0, None, None]]
    def generate_onePeriod_equilibrium(self):
        """
        return a tuple of (knext, w, R)
        and track the dynamics in self.equilibrium_dynamics
        """ 
        alpha=self.producer.alpha
        beta=self.consumer.beta

        w=(1-alpha)*self.k**alpha
        R=alpha*self.k**(alpha-1)
        knext=(beta/(1+beta))*w

        # keep track of the dynamics
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][1] = w
        self.equilibrium_dynamics[len(self.equilibrium_dynamics)-1][2] = R
        self.equilibrium_dynamics.append([knext, None, None])

        # set the market time to the next period
        self.k = knext
        return (knext, w, R)
    
    def clear_equilibrium_dynamics(self):
        self.equilibrium_dynamics = [[self.k0, None, None]]
        self.k = self.k0

consumer = Consumer(0.9)
producer = Producer(0.3)
market = OLGMarket(consumer, producer, 1)

In [19]:
# initial state
market.k0, market.equilibrium_dynamics

(1, [[1, None, None]])

In [27]:
# dynamic equilibrium
market.generate_onePeriod_equilibrium()
market.k0, market.equilibrium_dynamics

(1,
 [[1, 0.7, 0.3],
  [0.33157894736842103, 0.5026597658109342, 0.6496962959461055],
  [0.23810199433149515, 0.4551210785224081, 0.8191946957141344],
  [0.21558366877377225, 0.4417563465167363, 0.8781933695814612],
  [0.20925300624476983, 0.4378239873111994, 0.8967080333618369],
  [0.2073903097789892, 0.4366511196537161, 0.9023381774043994],
  [0.20683474088860238, 0.436299872418994, 0.9040341037719941],
  [0.20666836061952348, 0.436194553355657, 0.9045435029644798],
  [0.20661847264215333, None, None]])

In [28]:
# simulate the dynamics
for t in range(30):
    _ = market.generate_onePeriod_equilibrium()

market.equilibrium_dynamics

[[1, 0.7, 0.3],
 [0.33157894736842103, 0.5026597658109342, 0.6496962959461055],
 [0.23810199433149515, 0.4551210785224081, 0.8191946957141344],
 [0.21558366877377225, 0.4417563465167363, 0.8781933695814612],
 [0.20925300624476983, 0.4378239873111994, 0.8967080333618369],
 [0.2073903097789892, 0.4366511196537161, 0.9023381774043994],
 [0.20683474088860238, 0.436299872418994, 0.9040341037719941],
 [0.20666836061952348, 0.436194553355657, 0.9045435029644798],
 [0.20661847264215333, 0.43616296259444404, 0.9046963786862854],
 [0.20660350859736823, 0.4361534858122312, 0.9047422464409072],
 [0.2065990195952674, 0.43615064281771954, 0.9047560072207563],
 [0.20659767291365663, 0.43614978992297976, 0.9047601354955237],
 [0.20659726891088517, 0.436149534054883, 0.9047613739816269],
 [0.20659714771020773, 0.4361494572944833, 0.9047617455277887],
 [0.2065971113500184, 0.436149434266366, 0.9047618569916669],
 [0.20659710044196286, 0.436149427357931, 0.904761890430833],
 [0.20659709716954627, 0.43614

In [ ]:
# Overlapping generation model
# Consumer: CRRA preferences
class Consumer:
    def __init__(self, beta):
        self.beta=beta
    def foc_c(self, knext, w):
        """
        s=knext
        """
        return knext-(self.beta/(1+self.beta))*w
# Producer: Cobb-Douglas
class Producer:
    def __init__(self, alpha):
        self.alpha=alpha
    def foc_l(self, k, w, l=1):
        return (1-self.alpha)*(k/l)**(self.alpha)-w
    def foc_k(self, k, R, l=1):
        return self.alpha*(l/k)**(1-self.alpha)-R